In [1]:
import requests
from  bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
from random import random
import sqlite3

## Работа с sqlite3

In [2]:
con = sqlite3.connect('quotes.db')

In [3]:
cursor = con.cursor()

In [4]:
cursor.execute('''create table if not exists quotes (
    disc TEXT,
    author varchar(200),
    author_link text,
    tags text)''')
con.commit()

In [5]:
def select_sqlite3(table, cursor):
    """
    table - Название таблицы (строка)
    cursor - курсор БД sqlite3
    """
    data = f'''select * from {table}'''
    column_names = f'PRAGMA table_info({table})'
    return pd.DataFrame([i for i in cursor.execute(data)], columns=[i[1] for i in cursor.execute(column_names)])

In [6]:
def insert_sqlite3(data, cursor, table):
    
    """
    data - Передаем список словарей (список словарей)
    cursor - курсор БД sqlite3
    table - Название таблицы (строка)
    """
    cursor.executemany(
    f'''insert into {table} {tuple(all_list[0].keys())}
    values(
        ?, ?, ?, ?
    )
    ''', [tuple(i.values()) for i in data])
    print('insert_successfully')

## Работа с sqlite3 Конец

In [7]:
url = 'https://quotes.toscrape.com/'

In [8]:
all_list = []
while True:
    r = requests.get(url)
    
    if r.status_code != 200:
        print(r.status_code, url)
        break
    soup = bs(r.content, 'lxml')
    card = soup.find_all('div', class_='quote')
    print(f'\n{url}\n')
    for i, item in enumerate(card, 1):
        a_temp = {'disc': '', 'author': '', 'author_link': '', 'tags': ''}
        a_temp['disc'] = item.find('span', class_='text').text
        a_temp['author'] = item.find('small', class_='author').text
        a_temp['author_link'] = 'https://quotes.toscrape.com' + item.find('small', class_='author').find_next().get('href')
        a_temp['tags'] = item.find('meta').get('content')
        all_list.append(a_temp)
        print(f'Цитата {i} Done')
        sleep(round(random(), 2))
    insert_sqlite3(all_list, cursor, 'quotes')
    all_list = []
    try:
        page_ = soup.find('li', class_='next').find('a').get('href')
        url = 'https://quotes.toscrape.com' + page_
    except AttributeError:
        print('ALL_Done\n')
        break


https://quotes.toscrape.com/

Цитата 1 Done
Цитата 2 Done
Цитата 3 Done
Цитата 4 Done
Цитата 5 Done
Цитата 6 Done
Цитата 7 Done
Цитата 8 Done
Цитата 9 Done
Цитата 10 Done
insert_successfully

https://quotes.toscrape.com/page/2/

Цитата 1 Done
Цитата 2 Done
Цитата 3 Done
Цитата 4 Done
Цитата 5 Done
Цитата 6 Done
Цитата 7 Done
Цитата 8 Done
Цитата 9 Done
Цитата 10 Done
insert_successfully

https://quotes.toscrape.com/page/3/

Цитата 1 Done
Цитата 2 Done
Цитата 3 Done
Цитата 4 Done
Цитата 5 Done
Цитата 6 Done
Цитата 7 Done
Цитата 8 Done
Цитата 9 Done
Цитата 10 Done
insert_successfully

https://quotes.toscrape.com/page/4/

Цитата 1 Done
Цитата 2 Done
Цитата 3 Done
Цитата 4 Done
Цитата 5 Done
Цитата 6 Done
Цитата 7 Done
Цитата 8 Done
Цитата 9 Done
Цитата 10 Done
insert_successfully

https://quotes.toscrape.com/page/5/

Цитата 1 Done
Цитата 2 Done
Цитата 3 Done
Цитата 4 Done
Цитата 5 Done
Цитата 6 Done
Цитата 7 Done
Цитата 8 Done
Цитата 9 Done
Цитата 10 Done
insert_successfully

https://quo

In [9]:
select_sqlite3(cursor=cursor, table='quotes')

,disc,author,author_link,tags
0,“The world as we have created it is a process ...,Albert Einstein,https://quotes.toscrape.com/author/Albert-Eins...,"change,deep-thoughts,thinking,world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,https://quotes.toscrape.com/author/J-K-Rowling,"abilities,choices"
2,“There are only two ways to live your life. On...,Albert Einstein,https://quotes.toscrape.com/author/Albert-Eins...,"inspirational,life,live,miracle,miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,https://quotes.toscrape.com/author/Jane-Austen,"aliteracy,books,classic,humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,https://quotes.toscrape.com/author/Marilyn-Monroe,"be-yourself,inspirational"
...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,https://quotes.toscrape.com/author/Harper-Lee,better-life-empathy
96,“You have to write the book that wants to be w...,Madeleine L'Engle,https://quotes.toscrape.com/author/Madeleine-L...,"books,children,difficult,grown-ups,write,write..."
97,“Never tell the truth to people who are not wo...,Mark Twain,https://quotes.toscrape.com/author/Mark-Twain,truth
98,"“A person's a person, no matter how small.”",Dr. Seuss,https://quotes.toscrape.com/author/Dr-Seuss,inspirational


In [10]:
con.close()